# Basic building blocks

Get to know the basic building blocks of Superlinked.

1. Describe your data using Python classes with the [@sl.schema](https://github.com/superlinked/superlinked/blob/main/docs/reference/common/schema/schema.md) decorator or inherit your class from the [Schema](https://github.com/superlinked/superlinked/blob/main/docs/reference/common/schema/schema.md) base class.
2. Describe your vector embeddings from building blocks with [Spaces](https://github.com/superlinked/superlinked/blob/main/docs/reference/dsl/space/index.md).
3. Combine your embeddings into a queryable [Index](https://github.com/superlinked/superlinked/blob/main/docs/reference/dsl/index/index.m.md).
4. Define your search with dynamic parameters and weights as a [Query](https://github.com/superlinked/superlinked/blob/main/docs/reference/dsl/query/query.md).
5. Load your data using a [Source](https://github.com/superlinked/superlinked/blob/main/docs/reference/dsl/source/index.md).
6. Run your configuration with an [Executor](https://github.com/superlinked/superlinked/blob/main/docs/reference/dsl/executor/in_memory/in_memory_executor.md).

In [1]:
%pip install superlinked==36.3.1

In [2]:
from superlinked import framework as sl

Create a schema for your data.

In [3]:
class ParagraphSchema(sl.Schema):
    body: sl.String
    id: sl.IdField


# You can also define it the following way,
# but it might not work with strict type checkers.

# @sl.schema
# class ParagraphSchema:
#     body: sl.String
#     id: sl.IdField

Instantiate a new instance of your schema to start the pipeline definition.

In [4]:
paragraph = ParagraphSchema()

Create a space that will run a transformers model on the body of the paragraph.

In [5]:
relevance_space = sl.TextSimilaritySpace(text=paragraph.body, model="all-MiniLM-L6-v2")

Group your space in an index to make it retrievable.

In [6]:
paragraph_index = sl.Index(relevance_space)

Define a query that will search for similar paragraphs in the index. The parameters will be filled later on.

In [7]:
query = sl.Query(paragraph_index).find(paragraph).similar(relevance_space, sl.Param("query_text")).select_all()

Create an in-memory source and executor to try out your configuration

In [8]:
source: sl.InMemorySource = sl.InMemorySource(paragraph)
executor = sl.InMemoryExecutor(sources=[source], indices=[paragraph_index])
app = executor.run()

Insert some example data.

In [9]:
source.put([{"id": "happy_dog", "body": "That is a happy dog"}])
source.put([{"id": "happy_person", "body": "That is a very happy person"}])
source.put([{"id": "sunny_day", "body": "Today is a sunny day"}])

Query your data.

In [10]:
result = app.query(query, query_text="This is a happy person")

sl.PandasConverter.to_pandas(result)

,body,id,similarity_score
0,That is a very happy person,happy_person,0.802860
1,That is a happy dog,happy_dog,0.635954
2,Today is a sunny day,sunny_day,0.295396


Check how a different query can produce different results.

In [11]:
result = app.query(query, query_text="This is a happy dog")

sl.PandasConverter.to_pandas(result)

,body,id,similarity_score
0,That is a happy dog,happy_dog,0.899728
1,That is a very happy person,happy_person,0.477175
2,Today is a sunny day,sunny_day,0.275726
